In [ ]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `hf_token` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/h

# Experiments

In [ ]:
import pandas as pd

datasets = ['multiArith1','gsm8k','svamp']
def run_datasets(data):
  # for i in datasets:
  if data == 'gsm8k':

    splits = {'train': 'main/train-00000-of-00001.parquet', 'test': 'main/test-00000-of-00001.parquet'}
    df = pd.read_parquet("hf://datasets/openai/gsm8k/" + splits["train"]).head(100)



#############
  elif data=='multiArith1':
    df = pd.read_excel('data/multiArith_d0_l3.xlsx')
  
  elif data == "svamp":

    df = pd.read_parquet("hf://datasets/tongyx361/svamp/data/test-00000-of-00001.parquet").head(100)
    df["question"] = df["Body"] + " " + df["Question"]
    df = df.rename(columns={"Answer":"answer"})



  return df

In [ ]:
datasets

['gsm8k']

In [ ]:
# Define the full AoU pipeline (all three phases, callable from a single function)


from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple
import re
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    GenerationConfig,
)
def llm(messages_or_prompt: List[Dict] | str, stop: Optional[List[str]] = None) -> str:
    """Unified interface: chat models use chat template; others use plain prompt."""
    if CHAT_MODE:
        assert isinstance(messages_or_prompt, list), "For chat models, pass a list of role/content dicts."
        prompt = tokenizer.apply_chat_template(messages_or_prompt, tokenize=False, add_generation_prompt=True)
    else:
        assert isinstance(messages_or_prompt, str), "For non-chat models, pass a raw string prompt."
        prompt = messages_or_prompt

    input_ids = tokenizer(prompt, return_tensors="pt").to(model.device)
    output_ids = model.generate(**input_ids, generation_config=gen_cfg)
    text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Return only the assistant/completion part
    if CHAT_MODE:
        # naive split: take the text after the last "assistant" marker if present
        parts = re.split(r"(?:assistant|Assistant):", text, flags=re.IGNORECASE)
        return parts[-1].strip() if len(parts) > 1 else text.strip()
    else:
        return text[len(prompt):].strip() if text.startswith(prompt) else text.strip()

# Prompt templates
AOU_PHASE1_PROMPT = """You are given a question. Your job is to understand it and extract what is needed to solve it.

Step 1: Rephrase the question in your own words to clarify what it's asking.
Step 2: List the knowledge, facts, or steps required to answer it. Label each item with:
  - (GIVEN) if it is directly stated in the question
  - (INFERRED) if it is not stated but can be reasonably inferred
  - (MISSING) if it is not known or cannot be assumed

Use format:
Rephrased: ...
Needs:
G1: (GIVEN) ...
G2: (INFERRED) ...
G3: (MISSING) ...

Question:
{question}
"""

AOU_PHASE2_PROMPT = """You are given a list of items that were identified as needed to answer a question.

Please check each item and determine:
- Is it truly supported by the question, common knowledge, or logic?
- If INFERRED: is the inference safe?
- If MISSING: is it critical to solve the task?

Update each item as either:
  - (CONFIRMED): safe to use
  - (REJECTED): unsafe, hallucinated, or unsupported

Use format:
G1: (CONFIRMED) ...
G2: (CONFIRMED/REJECTED) ...
...

Original Question:
{question}

Original Needs:
{phase1_output}
"""

AOU_PHASE3_PROMPT = """Use the confirmed information below to answer the question.

Rules:
- Use only the CONFIRMED items
- Each reasoning step must cite the Gx it uses (e.g., "From G2...")
- If needed information is REJECTED, state a conditional ("If Gx were true, then...")

Question:
{question}

Confirmed Inputs:
{confirmed_info}

Answer:"""

# Run AoU pipeline
def aou_pipeline(question: str, verbose: bool = True) -> Dict:
    # --- Phase 1: Extract needs ---
    phase1_prompt = AOU_PHASE1_PROMPT.format(question=question)
    phase1_output = llm([{"role": "user", "content": phase1_prompt}]) if CHAT_MODE else llm(phase1_prompt)

    # --- Phase 2: Validate needs ---
    phase2_prompt = AOU_PHASE2_PROMPT.format(question=question, phase1_output=phase1_output)
    phase2_output = llm([{"role": "user", "content": phase2_prompt}]) if CHAT_MODE else llm(phase2_prompt)

    # --- Phase 3: Solve using validated info ---
    phase3_prompt = AOU_PHASE3_PROMPT.format(question=question, confirmed_info=phase2_output)
    final_output = llm([{"role": "user", "content": phase3_prompt}]) if CHAT_MODE else llm(phase3_prompt)

    if verbose:
        print("=== QUESTION ===")
        print(question)
        print("\n=== PHASE 1: UNDERSTANDING & NEEDS EXTRACTION ===")
        print(phase1_output)
        print("\n=== PHASE 2: VALIDATION OF NEEDS ===")
        print(phase2_output)
        print("\n=== PHASE 3: CONSTRAINED REASONING ===")
        print(final_output)

    return {
        "phase1": phase1_output,
        "phase2": phase2_output,
        "phase3": final_output
    }


In [ ]:
from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple
import re
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    GenerationConfig,
)
import os
import time
################
models = ['mistralai/Mistral-7B-Instruct-v0.3','microsoft/Phi-3.5-mini-instruct','deepseek-ai/deepseek-llm-7b-chat']
for m in models:
  print(f'---------------------STARTED EXPERIMENTS ON {m}')
  MODEL_NAME = m  # "meta-llama/Meta-Llama-3-8B-Instruct"
  DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

  DTYPE = torch.bfloat16 if torch.cuda.is_available() else torch.float32


  os.makedirs(f"outputs/{MODEL_NAME.split('/')[0]}", exist_ok=True)

  tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
  model = AutoModelForCausalLM.from_pretrained(
      MODEL_NAME,
      torch_dtype=DTYPE,
      device_map="auto" if DEVICE == "cuda" else None
  )

  gen_cfg = GenerationConfig(
      max_new_tokens=512,
      temperature=0.2,
      top_p=0.95,
      do_sample=True,
  )

  def is_chat_model(tok) -> bool:
    try:
        _ = tok.apply_chat_template([{"role": "user", "content": "test"}], tokenize=False)
        return True
    except Exception:
        return False

  CHAT_MODE = is_chat_model(tokenizer)

  for i in datasets:
    print(f'------------- starting {i}---------')
    # file_path = "results.xlsx"
    x = run_datasets(i)

    file_path = f"outputs/{MODEL_NAME.split('/')[0]}/{i}.xlsx"
    # path = "my_directory"

# Make dir if not exists

    for q in range(len(x)):
      if __name__ == "__main__":
        start_time = time.time()
        answer = aou_pipeline(x['question'][q], verbose = False)
        final = answer['phase3']
        runtime_secs = round(time.time() - start_time, 2)
        # final = answer['final'].split("Now produce your final response:")[-1]

        new_data = {'question':[x['question'][q]],'ground_truth':[x['answer'][q]],'predicted_answer':[final], 'runtime':[runtime_secs]}
        df_new = pd.DataFrame(new_data)

        # file_path = "results.xlsx"

        if os.path.exists(file_path):
            existing_df = pd.read_excel(file_path, sheet_name="Sheet1")
            # Open existing workbook and append
            if x['question'][q] not in existing_df["question"].values:
              with pd.ExcelWriter(file_path, mode="a", engine="openpyxl", if_sheet_exists="overlay") as writer:
                  # Find next row

                  startrow = len(existing_df) + 1
                  df_new.to_excel(writer, sheet_name="Sheet1", index=False, header=False, startrow=startrow)
        else:
            # Create new file
            df_new.to_excel(file_path, sheet_name="Sheet1", index=False)
        print(q)
      print(f'------------- finished {i}---------')










---------------------STARTED EXPERIMENTS ON deepseek-ai/deepseek-llm-7b-chat


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

...

97
------------- finished gsm8k---------


Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


98
------------- finished gsm8k---------


Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


99
------------- finished gsm8k---------


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# if __name__ == "__main__":

#   q = "The Berlin Wall fell in 1989. Alice was born that year and is now 30. What year is it?"
#   aap_pipeline(q)